In [1]:
# Only needs to run if not already installed.
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8e43e571e1b9cdf1ab4586acfbf554dccb35497d64e1e92d71bb15b45a1aa42c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
# Import data from Excel workbook.
# There should be two sheets (Sheet1, Sheet2) with headings id & description.

import pandas as pd

DataFile = "LearningOutcomesSheet.xlsx"

df1 = pd.read_excel(DataFile, 'Sheet1')
df2 = pd.read_excel(DataFile, 'Sheet2')

# Add the ids and text chunks to lists.
Sheet1_id = df1.id.values.tolist()
Sheet1_text = df1.description.values.tolist()
Sheet2_id = df2.id.values.tolist()
Sheet2_text = df2.description.values.tolist()

In [3]:
import os

print(os.path.split(DataFile)[0])

In [4]:
Sheet1_id = df1.id.values.tolist()
Sheet1_text = df1.description.values.tolist()
Sheet2_id = df2.id.values.tolist()
Sheet2_text = df2.description.values.tolist()
#Sheet2_id

In [5]:
from sentence_transformers import SentenceTransformer

# Models - https://huggingface.co/models?library=sentence-transformers
model = SentenceTransformer('all-mpnet-base-v2')

In [6]:
embeddings1 = model.encode(Sheet1_text)
embeddings2 = model.encode(Sheet2_text)

In [7]:
from sentence_transformers.util import cos_sim

scores = cos_sim(embeddings1, embeddings2)
#scores

In [8]:
import numpy as np

scores_df = pd.DataFrame(scores.numpy(), index = Sheet1_id, columns = Sheet2_id)
scores_df
transposed_df = scores_df.transpose()
#transposed_df

In [9]:
print(len(scores_df))
print(len(transposed_df))
print(scores_df.shape)

94
204
(94, 204)


In [10]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.4 MB/s eta 0:00:00


In [11]:
#outcome_subcomp_df.to_excel(r'C:\\Users\\u0396993\\Documents\\junk\\Outcomes Competencies CosignSimilarityMatrix.xlsx')


# Set the path and name for the Excel workbook to create.
#OutputFile = "C:\\Users\\u0396993\\Documents\\junk\\BYU_LearningOutcomesSheet _NONPF_CosignSimilarityReport_3.xlsx"
OutputFile = os.path.split(DataFile)[0] + "\\CosineSimilarity_" + os.path.split(DataFile)[1]


# Determine column letters from column numbers
# https://stackoverflow.com/questions/29351492/how-to-make-a-continuous-alphabetic-list-python-from-a-z-then-from-aa-ab-ac-e
def char_label(n, chars):
    indexes = []
    while n:
        residual = n % len(chars)
        if residual == 0:
            residual = len(chars)
        indexes.append(residual)
        n = (n - residual)
        n = n // len(chars)
    indexes.reverse()
    label = ''
    for i in indexes:
        label += chars[i-1]
    return label

my_chrs = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
#start_col = 2
#end_col = start_col + 204
#start_row = 10
#end_row = start_row + 27
#for col in range(start_col, end_col+1):
#    r=char_label(col, my_chrs)
#    print(r + str(1) + " = max(" + r + str(start_row) + ":" + r + str(end_row) + ")")

# Use a with statement to close the file automatically when all file writing and formatting is complete.
with pd.ExcelWriter(OutputFile) as writer:

    # Write the scores dataframe to the Excel workbook. Leave blank rows at the top.
    scores_df.to_excel(writer, sheet_name="Sheet1", startrow=10, startcol=0)

    # Get the number of rows and columns
    (max_row, max_col) = scores_df.shape

    # Load the sheet as an object to do some formatting later
    sheet1 = writer.sheets['Sheet1']

    # Label the first row as "Max". I'll identify the highest match in Excel later.
    sheet1.write(0,0,"Max")

    # Add formula for Max value in column in first row.
    for col in range(2, max_col+2):
        r=char_label(col, my_chrs)
        sheet1.write(0, col-1, "= max(" + r + str(12) + ":" + r + str(11+max_row) + ")")

    sheet1.conditional_format(0,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#008000", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    # Set up conditional formatting in the worksheet.
    sheet1.conditional_format(10,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#0000FF", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    # Add the description text as comments for the rows.
    for i in range(len(Sheet1_text)):
        sheet1.write_comment(i+11,0,Sheet1_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})

    # Add the description text as comments for the columns.
    for i in range(len(Sheet2_text)):
        sheet1.write_comment(10,i+1,Sheet2_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})

    # Add the transposed data to a second sheet with same formatting as first sheet.
    transposed_df.to_excel(writer, sheet_name="Sheet2", startrow=10, startcol=0)
    (max_row, max_col) = transposed_df.shape
    sheet2 = writer.sheets['Sheet2']
    sheet2.write(0,0,"Max")

    # Add formula for Max value in column in first row.
    for col in range(2, max_col+2):
        r=char_label(col, my_chrs)
        sheet2.write(0, col-1, "= max(" + r + str(12) + ":" + r + str(11+max_row) + ")")

    sheet2.conditional_format(0,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#008000", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})

    sheet2.conditional_format(10,1, max_row+10, max_col, {"type": "3_color_scale", "mid_color": "#FFFFFF", "max_color": "#0000FF", "min_color": "#FFFF00", "min_value": -1, "max_value": 1, "mid_value": 0})
    for i in range(len(Sheet2_text)):
        sheet2.write_comment(i+11,0,Sheet2_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})
    for i in range(len(Sheet1_text)):
        sheet2.write_comment(10,i+1,Sheet1_text[i], {'text_wrap': 0, "x_scale": 2, "y_scale": 2})
print("Done")

Done
